In [36]:
from stable_baselines3 import PPO
from myosuite.utils import gym as myo_gym

from config import Config

import numpy as np

In [32]:
cfg = Config()

env = myo_gym.make(cfg.env_id)
obs, info = env.reset()


In [33]:
action = env.action_space.sample()

obs, reward, terminated, truncated, info = env.step(action)


In [37]:
obs_dict = env.obs_dict
pelvis_pos = np.asarray(obs_dict["pelvis_pos"], dtype=np.float32)

/home/ubuntu/myochallenge_2025eval/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.obs_dict to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.obs_dict` for environment variables or `env.get_wrapper_attr('obs_dict')` that will search the reminding wrappers.
  logger.warn(


In [38]:
pelvis_pos

array([ 2.0201e+00, -1.0802e-03,  9.5000e-01], dtype=float32)

In [35]:
info['done']

0